In [ ]:
#Fine-tuning the whisper model
import os
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
csv_path = r'C:\Users\sarve\Downloads\Dataset1\Recordings\audio__details.csv'
audio_folder_path = r'C:\Users\sarve\Downloads\Dataset1\Recordings\train'

df = pd.read_csv(csv_path)

df['file_name'] = df['file_name'].apply(lambda x: os.path.abspath(os.path.join(audio_folder_path, os.path.basename(x))))
df = df[df['file_name'].apply(os.path.exists)]
df.to_csv(csv_path, index=False)

dataset = load_dataset('csv', data_files=csv_path)
dataset = dataset.cast_column('file_name', Audio(sampling_rate=16000))
dataset = dataset.rename_column('file_name', 'audio')
dataset = dataset.rename_column('phrase', 'sentence')

processor = WhisperProcessor.from_pretrained("openai/whisper-base")

def prepare_dataset(batch):
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch
dataset = dataset.map(prepare_dataset)
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-finetuned1",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=False,
    save_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
)
class DataCollatorForWhisper:
    def __call__(self, features):
        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        labels = [torch.tensor(feature["labels"]) for feature in features]
        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_features": input_features_padded,
            "labels": labels_padded
        }

data_collator = DataCollatorForWhisper()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
)
trainer.train()
model.save_pretrained("whisper-finetuned0")
processor.save_pretrained("whisper-finetuned0")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

C:\Users\sarve\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\sarve\AppData\Local\Temp\ipykernel_20128\1178619654.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
